---
title: ""
---

In [ ]:
#| echo: false
#| output: asis

import pandas as pd

from cnvsapi.build_utils import get_current_week_from_lecture_id, get_outline_objects, extract_outline
from datetime import datetime, timedelta
from cnvsapi.config import config

output_file = "./detailed-gantt.mmd"


outline_df = extract_outline().copy()
outline_df.loc[:,'date'] = pd.to_datetime(outline_df['DateCanvas']).dt.date

objects_df = get_outline_objects().copy()
objects_df.loc[:,'canvas_due_at'] = pd.to_datetime(objects_df['canvas_due_at']).dt.date
objects_df = objects_df.dropna(subset=["canvas_due_at"])

summary_df = outline_df.groupby('Module').agg(
    earliest_date=('date', 'min'),
    latest_date=('date', 'max')
).reset_index()
summary_df = summary_df.sort_values(by='earliest_date').reset_index(drop=True)

# Find the row(s) where "Notes, Lectures" contains "Classes begin"
matching_rows = outline_df[outline_df['Notes, Lectures'].str.lower().str.contains('classes begin', na=False)]
first_date = pd.to_datetime(matching_rows['date'].iloc[0]).date()  - timedelta(days=1)
if first_date is not None:
    summary_df.at[0, 'earliest_date'] = first_date

matching_rows = outline_df[outline_df['Notes, Lectures'].str.lower().str.contains('last day of classes', na=False)]
last_date = pd.to_datetime(matching_rows['date'].iloc[0]).date()
if last_date is not None:
    summary_df.at[summary_df.index[-1], 'latest_date'] = last_date
# Add one day to the latest dates to account for the 24-hours of the last day
summary_df['latest_date'] = summary_df['latest_date'].apply(lambda x: x + timedelta(days=1))

course = config.get_default_course()
course_title = course['name']

In [ ]:
def clean_title( title ):
    return title.replace(":",".")


def make_gantt():
    """ make gant chart """
    # VCU gold: #FFB300

        # Define the content you want to write to the file
    content = []

    # Start writing the header
    content.append(f"""gantt
    title {course_title}
    dateFormat YYYY-MM-DD
    axisFormat %b %e
    todayMarker stroke-width:7px,stroke:#FFB300,opacity:0.5
"""
    )

    for i,item in summary_df.iterrows():
        deliverables_df = objects_df[ (objects_df['module']==item['Module']) & (objects_df['canvas_assignment_group']!="Projects") ]
        deliverables_df = objects_df[ (objects_df['module']==item['Module']) & (objects_df['canvas_assignment_group']!="Projects") ]

 #       if not deliverables_df.empty:
        deliverables_df = deliverables_df.sort_values(by='canvas_due_at').reset_index(drop=True)
        content.append(f"    section {item['Module']}")
        content.append(f"    {item['Module']} : m{i}, {item['earliest_date']}, {item['latest_date']}")
        for j,row in deliverables_df.iterrows():
            content.append(f"    {clean_title(row['outline_entry'])} : milestone, {row['canvas_due_at']}, 0d")
        content.append("")

    projects_df = objects_df[ (objects_df['canvas_assignment_group']=="Projects") ]
    if not projects_df.empty:
        content.append(f"    section Semester Project")
        for j,row in projects_df.sort_values(by='canvas_due_at').reset_index(drop=True).iterrows():
            content.append(f"    {clean_title(row['outline_entry'])} : milestone, {row['canvas_due_at']}, 0d")
        content.append("")


    # Open the file in write mode ('w' will create the file if it doesn't exist or overwrite it if it does)
    with open(output_file, "w") as file:
        file.write("\n".join(content))

make_gantt()

::: {.no-filter}

```{mermaid}
%%| echo: false
%%| file: ./detailed-gantt.mmd
```

:::